In [ ]:
import numpy as np
from pathlib import Path
import re
import ast

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
DATA_DIR = Path("../data")
RAW_DATA_DIR = DATA_DIR / "raw"
PROCESSED_DATA_DIR = DATA_DIR / "processed"